In [1]:
print("ok")

ok


In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [17]:
prompt = ChatPromptTemplate.from_template("tell me a achivemet and award  fact about {soccer_player}")

output_parser = StrOutputParser()

In [18]:
chain = prompt | model | output_parser

In [19]:

chain.invoke({"soccer_player": "Ronaldo"})

"One achievement of Cristiano Ronaldo is that he has won the prestigious FIFA Ballon d'Or award five times, in 2008, 2013, 2014, 2016, and 2017. This award is given annually to the best male footballer in the world, based on votes from national team coaches and captains, as well as journalists from around the world. Ronaldo's five Ballon d'Or wins make him one of the most decorated players in the history of football."

## Use of .bind() to add arguments to a Runnable in a LCEL Chain
* For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [15]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser

In [16]:
chain.invoke({"soccer_player": "Ronaldo"})

'One of Cristiano '

In [20]:
prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {historical_figure}.")

In [23]:
chain = prompt | model.bind(stop=["Israel"]) | StrOutputParser()

In [24]:
result = chain.invoke({"historical_figure": "Albert Einstein"})
print(result) 

Albert Einstein was offered the presidency of 


# Combine chain


In [25]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | model | StrOutputParser()

In [27]:
chain.invoke("modi")

'Narendra Modi is the current Prime Minister of India and a prominent figure in Indian politics.'

In [28]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

In [29]:
composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [30]:
composed_chain.invoke({"politician": "modi"})

"It is subjective to determine whether Narendra Modi's leadership and initiatives are positive for humanity. Some may argue that his focus on economic development has led to growth and prosperity in India, benefiting millions of people. Others may criticize his government's handling of social issues, human rights concerns, and environmental policies. Ultimately, the impact of Modi's leadership on humanity will depend on individual perspectives and the outcomes of his policies in the long run."

In [33]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI



prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)


model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

In [34]:
chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

In [38]:
chain2.invoke({"politician": "trump", "language": "bhojpuri"})

'Donald Trump ke des United States ke North America me hai.'

# RAG Applicaiton//


In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

In [6]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits=text_splitter.split_documents(docs)

In [7]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [8]:
retriever = vectorstore.as_retriever()

In [9]:
prompt = hub.pull("rlm/rag-prompt")

/Users/myhome/Downloads/LANG/lang/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [18]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [19]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique where complex tasks are broken down into smaller and simpler steps to make them more manageable. It can be done using prompting techniques like Chain of Thought or Tree of Thoughts to guide the model in decomposing tasks effectively. Task decomposition can be facilitated by the use of LLM with simple prompts, task-specific instructions, or human inputs.'